In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from prettytable import PrettyTable
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
# Lets look at the advertising data set
ad_df = pd.read_csv('datasets/advertising.csv')
ad_df.head()

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,12.0
3,151.5,41.3,58.5,16.5
4,180.8,10.8,58.4,17.9


---------------------

### Create different multi predictor models

In [3]:
# Initialize a list to store the MSE values
mse_list = []

# Create a list of lists of all unique predictor combinations
# For example, if you have 2 predictors, A and B, you would end up with:
# [['A'], ['B'], ['A', 'B']]
cols = [['TV'], ['Radio'], ['Newspaper'], ['TV', 'Radio'], ['TV', 'Newspaper'], 
        ['Radio', 'Newspaper'], ['TV', 'Radio', 'Newspaper']]

# Loop over all predictor combinations:
for i in cols:

    # Set each of the predictors from the previous list as x:
    x = ad_df[i]

    # Set the "sales" column as the response variable:
    y = ad_df['Sales']

    # Split the data into train-test sets with 80% training data and 20% testing data.
    # Set random_state as 0
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, random_state = 0)

    # Initialize a Linear Regression model
    lreg = LinearRegression()

    # Fit the linear model on the train data:
    lreg.fit(x_train, y_train)

    # Predict the response variable for the test set using the trained model
    y_pred = lreg.predict(x_test)

    # Compute the MSE for the test data:
    MSE = mean_squared_error(y_test, y_pred)

    # Append the computed MSE to the initialized list:
    mse_list.append(MSE)

In [4]:
# Take a quick look at the mse_list to ensure it's populated:
mse_list

[6.639091226559303,
 30.800740055187482,
 34.972705918316024,
 4.476818366208145,
 5.93963320856206,
 31.445129067266254,
 4.522552562041294]

### Display the MSE with predictor combinations

In [5]:
# Helper code to display the MSE for each predictor combination:
t = PrettyTable(['Predictors', 'MSE'])

for i in range(len(mse_list)):
    t.add_row([cols[i], round(mse_list[i], 3)])

print(t)

+------------------------------+--------+
|          Predictors          |  MSE   |
+------------------------------+--------+
|            ['TV']            | 6.639  |
|          ['Radio']           | 30.801 |
|        ['Newspaper']         | 34.973 |
|       ['TV', 'Radio']        | 4.477  |
|     ['TV', 'Newspaper']      |  5.94  |
|    ['Radio', 'Newspaper']    | 31.445 |
| ['TV', 'Radio', 'Newspaper'] | 4.523  |
+------------------------------+--------+


In [6]:
ad_df

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,12.0
3,151.5,41.3,58.5,16.5
4,180.8,10.8,58.4,17.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,14.0
197,177.0,9.3,6.4,14.8
198,283.6,42.0,66.2,25.5


In [12]:
# check why output doesn't match edx 
# Sales sum is different in this data set so that is likely causing the disconnect
for i in ad_df.columns:
    print(ad_df[i].sum())

29408.5
4652.800000000001
6110.799999999999
3026.1000000000004
